## Криптографія - це дуже важлива річ для інформаційної безпеки. Інформаційна безпека складається з 4 частин:

# Цілісність: гарантує, що документ не буде змінено
# Конфіденційність: гарантує, що тільки уповноважені люди можуть читати документ
# Аутентифікація: переконайтеся, що документ був написаний ідентифікованою особою
# Невідворотність: довести, від кого/звідки прийшов документ, а також автентичність цього повідомлення, щоб відправник не зміг заперечити, що він його відправив.
# Давайте подивимось, що надає .NET для кожної частини!

# Генерація випадкових чисел
Генерація випадкових чисел є поширеним способом створення криптографічних ключів. Ці ключі повинні бути якомога більш випадковими, щоб їх неможливо було передбачити. Не використовуйте System.Random для генерації криптографічних чисел. Замість цього використовуйте System.Security.Cryptography.RNGCryptoServiceProvider.

In [34]:
static string GetRandomDataAsString(byte[] randomData)
{
    string hexString = BitConverter.ToString(randomData).Replace("-", "");
    return hexString;
}

In [35]:
// old versions of .NET Frameworks
byte[] GetRandomData(int length)
{
    using (var rngCsp = new System.Security.Cryptography.RNGCryptoServiceProvider())
    {
        var randomData = new byte[length];
        rngCsp.GetBytes(randomData);
        return randomData;
    }
}

byte[] randomData = GetRandomData(16); // Generate 16 bytes of random data
Console.WriteLine(GetRandomDataAsString(randomData));

E3DB84D2A18C125AE843C67F88B7B263


In [36]:
using System.Security.Cryptography;
// .NET Core 2.1+
static byte[] GetRandomData(int length)
{
    var randomData = new byte[length];
    RandomNumberGenerator.Fill(randomData);
    return randomData;
}

byte[] randomData = GetRandomData(16); // Generate 16 bytes of random data
Console.WriteLine(GetRandomDataAsString(randomData));

EE4F56805D9D48886B1CED603B60335F


In [14]:
using System.Security.Cryptography;
// .NET 6 and higher
byte[] GetRandomData(int length)
{
    return RandomNumberGenerator.GetBytes(length);
}
byte[] randomData = GetRandomData(16); // Generate 16 bytes of random data
Console.WriteLine(GetRandomDataAsString(randomData));

8A382A18DCA63DDEE3524A0C05127E73


# Генерація криптографічного ключа з пароля
Пароль має змінну довжину і часто складається лише з того, що користувач може ввести з клавіатури. У такому вигляді ви не можете використовувати його для шифрування даних. Дійсно, якщо ви хочете зашифрувати блок даних за допомогою AES 128, вам потрібен ключ фіксованої довжини 128 біт. Замість цього вам потрібно створити криптографічний ключ потрібного розміру з пароля користувача. Цим і займається "Функція виведення ключа на основі пароля 2" (PBKDF2). У .NET клас, який реалізує цей алгоритм, називається [Rfc2898DeriveBytes](https://learn.microsoft.com/en-us/dotnet/api/system.security.cryptography.rfc2898derivebytes?view=net-7.0&WT.mc_id=DT-MVP-5003978)

In [15]:
string password = "DenysTestPass";

byte[] salt;
using (var derivedBytes = new System.Security.Cryptography.Rfc2898DeriveBytes(password, saltSize: 16, iterations: 50000, HashAlgorithmName.SHA256))
{
    salt = derivedBytes.Salt;
    byte[] key = derivedBytes.GetBytes(16); // 128 bits key
    Console.WriteLine(Convert.ToBase64String(key)); // Qs117rioEMXqseslxc5X4A==
}

4Nt/+Ir3PcPUXTI/wsPhAg==


Ви можете відтворити той самий ключ, використовуючи той самий пароль, ту саму кількість ітерацій, ту саму сіль і той самий алгоритм. Це означає, що вам потрібно десь зберігати цю інформацію (крім пароля!), щоб мати можливість використовувати її пізніше.

In [16]:
byte[] key;
using (var derivedBytes = new Rfc2898DeriveBytes(password, salt, iterations: 50000, HashAlgorithmName.SHA256))
{
    key = derivedBytes.GetBytes(16); // Same as the first generated key
    Console.WriteLine(Convert.ToBase64String(key));
}

// .NET 6+
Rfc2898DeriveBytes.Pbkdf2(password, salt, iterations: 50000, HashAlgorithmName.SHA256, outputLength: 16);
Console.WriteLine(Convert.ToBase64String(key)); 

4Nt/+Ir3PcPUXTI/wsPhAg==
4Nt/+Ir3PcPUXTI/wsPhAg==


Якщо ви зміните один з параметрів, то отримаєте інший ключ:

In [17]:
// Number of iteration changed (49999 instead of 50000)
using (var derivedBytes = new Rfc2898DeriveBytes(password, salt, iterations: 49999, HashAlgorithmName.SHA256))
{
    byte[] key = derivedBytes.GetBytes(16);
    Console.WriteLine(Convert.ToBase64String(key));
}

7w/O5TW0Voqu8Q8ftw9lEA==


# Хеш-алгоритми
Хеш-алгоритми перетворюють двійкове повідомлення довільної довжини на менше двійкове значення фіксованої довжини. На один і той самий вхід повинен виходити один і той самий хеш. Якщо ви змінюєте один байт на вході, хеш повинен бути іншим. Якщо хеш криптографічно стійкий, його значення буде суттєво змінюватися. Хеш-алгоритм повинен мати наступні характеристики:

# Повинно бути неможливо згенерувати конкретний хеш
# Неможливо змінити повідомлення без зміни хешу
# Неможливо знайти 2 різні вхідні дані з однаковими хешами

Коли хтось хоче відправити дані комусь іншому, він може відправити дані і їх хеш. Отримувач може обчислити хеш отриманих даних і порівняти його зі значенням хешу, надісланого відправником. Таким чином одержувач може перевірити, що дані не були змінені під час передачі. Таким чином, ви можете перевірити цілісність даних.

![Alt text](image.png)

У .NET ви можете використовувати будь-який клас, успадкований від HashAlgorithm. Однак деякі алгоритми, такі як [MD5](https://www.kb.cert.org/vuls/id/836068/) або [SHA1](https://security.googleblog.com/2017/02/announcing-first-sha1-collision.html), вважаються вразливими, оскільки вони не враховують попередні характеристики. Ви можете використовувати будь-які алгоритми, похідні від [SHA2](https://en.wikipedia.org/wiki/SHA-2), такі як SHA256, SHA384 або SHA512. Алгоритми [SHA3](https://en.wikipedia.org/wiki/SHA-3) підтримуються в .NET 8.

In [18]:
using System.Security.Cryptography;
byte[] ComputeHash(byte[] data)
{
    using (var sha256 = SHA256.Create())
    {
        return sha256.ComputeHash(data);
    }
}

// .NET 5.0+
byte[] LambdaComputeHash(byte[] data) => SHA256.HashData(data);

# Автентифіковане хешування (HMAC)
Код автентифікації повідомлення на основі хешування (HMAC) можна використовувати для перевірки як цілісності даних, так і автентичності повідомлення. Він працює як класичний хеш-алгоритм, за винятком того, що вам потрібен секретний ключ для генерації та перевірки хешу: HMAC = хеш + криптографічний ключ. Тільки той, хто володіє секретним ключем, може згенерувати і перевірити хеш. Поки ключ секретний, ви знаєте, хто згенерував хеш.

HMAC дуже корисний, коли ви надсилаєте комусь повідомлення і очікуєте, що він поверне його назад. Тоді ви можете переконатися, що повідомлення дійсно було створено вами і не було змінено. Наприклад, ASP.NET WebForms надсилає клієнту стан перегляду та його HMAC. Коли клієнт надсилає POST-запит на сервер, він повертає значення ViewState. Сервер може перевірити, що HMAC ViewState був згенерований сервером і не був змінений клієнтом.

![Alt text](image-1.png)

# Попередження

Спільний доступ до ключа між клієнтами суперечить меті автентифікації, оскільки будь-хто зможе створити дійсний хеш з даним ключем. Подумайте про використання секрету, який знають тільки сервер і клієнт.

In [19]:
byte[] ComputeHmac256(byte[] key, byte[] data)
{
    using (var hmac = new HMACSHA256(key))
    {
        return hmac.ComputeHash(data);
    }
}

In [20]:

// .NET 6
byte[] ComputeHmac256(byte[] key, byte[] data)
{
    return HMACSHA256.HashData(key, data);
}

# Шифрування
Шифрування зашифровує вміст файлу, щоб тільки авторизовані користувачі могли його прочитати. Існує 2 способи шифрування даних. Перший відомий як симетричне шифрування. У цьому типі алгоритмів ключ для шифрування і розшифрування даних однаковий. Це означає, що вам потрібен безпечний спосіб передачі ключа іншим людям. Інший спосіб - асиметричне шифрування. Ключі для шифрування і дешифрування даних відрізняються. Ми часто використовуємо терміни "відкритий ключ" і "закритий ключ". Відкритий ключ дозволяє зашифрувати дані, щоб ви могли поділитися ними з будь-ким. Закритий ключ дозволяє розшифровувати дані, тому ви повинні зберігати його в таємниці.

Основним симетричним алгоритмом є AES, а RSA - найпоширеніший асиметричний алгоритм. AES дуже швидкий і може використовуватися з даними будь-якої довжини. RSA набагато повільніший і не може зашифрувати дані, довжина яких перевищує розмір ключа.

# Примітка

Шифрування не забезпечує цілісність даних. Вам потрібно використовувати хеш або HMAC, щоб переконатися, що дані, які ви розшифровуєте, не були змінені.


# Symmetric encryption


![Alt text](image-2.png)

# .NET надає декілька симетричних алгоритмів та декілька реалізацій для деяких алгоритмів:

[AES](https://en.wikipedia.org/wiki/Advanced_Encryption_Standard): AESManaged, AesCng, AesCryptoServiceProvider

[AES-GCM](https://en.wikipedia.org/wiki/Galois/Counter_Mode): AesGcm

[DES](https://en.wikipedia.org/wiki/Data_Encryption_Standard): DESCryptoServiceProvider

[RC2](https://en.wikipedia.org/wiki/RC2): RC2CryptoServiceProvider

[Rijndael](https://en.wikipedia.org/wiki/Advanced_Encryption_Standard): RijndaelManaged

[Triple_DES](https://en.wikipedia.org/wiki/Triple_DES): TripleDESCng, TripleDESCryptoServiceProvider

AES
Якщо ви не знаєте, який з них використовувати, ви можете почати з AES. AESManaged повністю реалізовано в .NET, однак ця реалізація не відповідає вимогам FIPS. AESCryptoServiceProvider використовує реалізацію Windows (CryptoAPI), яка сумісна з FIPS. Або ви можете створити екземпляр найкращого доступного постачальника за допомогою AES.Create().

Існує декілька режимів AES, кожен з яких має своє застосування. Ця відповідь на [Stack Overflow](https://stackoverflow.com/questions/1220751/how-to-choose-an-aes-encryption-mode-cbc-ecb-ctr-ocb-cfb/22958889#22958889) дуже добре описує кожен режим. Зверніть увагу, що деякі з них ще не реалізовані в .NET.

In [40]:
static void Main()
{
    var key = GetRandomData((256 / 8));

    var data = new byte[] { 1, 2, 3 };
    var encryptedData = Encrypt(data, key, out var iv);
    var decryptedData = Decrypt(encryptedData, key, iv);

    Console.WriteLine(GetRandomDataAsString(encryptedData));
    Console.WriteLine(GetRandomDataAsString(decryptedData));
}

static byte[] Encrypt(byte[] data, byte[] key, out byte[] iv)
{
    using var aes = Aes.Create();

    // You should adjust the mode depending on what you want to encrypt.
    // However, some mode may be weak or require additional security steps such as CBC: https://learn.microsoft.com/en-us/dotnet/standard/security/vulnerabilities-cbc-mode?WT.mc_id=DT-MVP-5003978
    aes.Mode = CipherMode.CBC;

    aes.Key = key;
    aes.GenerateIV(); // You must use a new IV for each encryption for security purpose

    using (var cryptoTransform = aes.CreateEncryptor())
    {
        iv = aes.IV;
        return cryptoTransform.TransformFinalBlock(data, 0, data.Length);
    }
}

static byte[] Decrypt(byte[] data, byte[] key, byte[] iv)
{
    using var aes = Aes.Create();
    aes.Key = key;
    aes.IV = iv;
    aes.Mode = CipherMode.CBC; // same as for encryption

    using (var cryptoTransform = aes.CreateDecryptor())
    {
        return cryptoTransform.TransformFinalBlock(data, 0, data.Length);
    }
}

Main();

690531B616FAD2F29B38233CECB9C2C2
010203


# AES-GCM (AEAD)
[AES-GCM](https://en.wikipedia.org/wiki/Galois/Counter_Mode) забезпечує конфіденційність і цілісність. Як і інші режими AES, він шифрує дані за допомогою ключа (конфіденційність). При розшифровці даних він гарантує, що повідомлення не було змінено (цілісність), чого не робиться в інших режимах. Ви можете розглядати його як безпечний спосіб поєднання AES і HMAC.

In [43]:
static void Main()
{
    var key = GetRandomData(256 / 8);

    var data = new byte[] { 1, 2, 3 };
    byte[]? associatedData = null; // Optional. If possible, provide something associated to the current context such as the current user id https://crypto.stackexchange.com/a/6716/16630
    var encryptedAesGcmData = EncryptAesGcm(data, key, associatedData, out var tag, out var nonce);
    var decryptedAesGcmData = DecryptAesGcm(encryptedAesGcmData, key, associatedData, tag, nonce);

    Console.WriteLine(GetRandomDataAsString(encryptedAesGcmData));
    Console.WriteLine(GetRandomDataAsString(decryptedAesGcmData));
}

public static byte[] EncryptAesGcm(byte[] data, byte[] key, byte[]? associatedData, out byte[] tag, out byte[] nonce)
{
    tag = new byte[AesGcm.TagByteSizes.MaxSize];
    nonce = new byte[AesGcm.NonceByteSizes.MaxSize];
    RandomNumberGenerator.Fill(nonce);

    byte[] cipherText = new byte[data.Length];
    using var cipher = new AesGcm(key);
    cipher.Encrypt(nonce, data, cipherText, tag, associatedData);
    return cipherText;
}

public static byte[] DecryptAesGcm(byte[] data, byte[] key, byte[]? associatedData, byte[] tag, byte[] nonce)
{
    byte[] decryptedData = new byte[data.Length];
    using var cipher = new AesGcm(key);
    cipher.Decrypt(nonce, data, tag, decryptedData, associatedData);
    return decryptedData;
}

Main();

4DACA8
010203


# ChaCha20Poly1305 (AEAD)
ChaCha20Poly1305 ([RFC 8439](https://datatracker.ietf.org/doc/html/rfc8439)) - це шифр [Authenticated Encryption with Associated Data (AEAD)](https://en.wikipedia.org/wiki/Authenticated_encryption), придатний для швидкої програмної реалізації, заснований на потоковому шифрі [ChaCha20](https://github.com/RustCrypto/stream-ciphers/tree/master/chacha20) та універсальній хеш-функції [Poly1305](https://github.com/RustCrypto/universal-hashes/tree/master/poly1305). ChaCha20Poly1305 є гарною альтернативою AES-GCM, коли машина не має апаратної підтримки AES. Дійсно, AES вразливий до [побічних каналів, заснованих на синхронізації](https://en.wikipedia.org/wiki/Advanced_Encryption_Standard#Side-channel_attacks), якщо це зроблено в програмному забезпеченні. Cloudflare написав два хороших пости про ChaCha20Poly1305: [It takes two to ChaCha (Poly)] (https://blog.cloudflare.com/it-takes-two-to-chacha-poly/), [Do the ChaCha: better mobile performance with cryptography.] (https://blog.cloudflare.com/do-the-chacha-better-mobile-performance-with-cryptography/)

In [ ]:
static void Main()
{
    var key = GetRandomData(32);

    var data = new byte[] { 1, 2, 3 };
    byte[]? associatedData = null; // Optional. If possible, provide something associated to the current context such as the current user id https://crypto.stackexchange.com/a/6716/16630
    var encryptedChaCha20Poly1305 = EncryptChaCha20Poly1305(data, key, associatedData, out var tag, out var nonce);
    var decryptedChaCha20Poly1305 = DecryptChaCha20Poly1305(encryptedChaCha20Poly1305, key, associatedData, tag, nonce);

    Console.WriteLine(GetRandomDataAsString(encryptedChaCha20Poly1305));
    Console.WriteLine(GetRandomDataAsString(decryptedChaCha20Poly1305));
}

public static byte[] EncryptChaCha20Poly1305(byte[] data, byte[] key, byte[]? associatedData, out byte[] tag, out byte[] nonce)
{
    tag = new byte[16];
    nonce = GetRandomData(12);
    byte[] cipherText = new byte[data.Length];
    using var cipher = new  ChaCha20Poly1305(key);
    cipher.Encrypt(nonce, data, cipherText, tag, associatedData);
    return cipherText;
}

public static byte[] DecryptChaCha20Poly1305(byte[] data, byte[] key, byte[]? associatedData, byte[] tag, byte[] nonce)
{
    byte[] decryptedData = new byte[data.Length];
    using var cipher = new ChaCha20Poly1305(key);
    cipher.Decrypt(nonce, data, tag, decryptedData, associatedData);
    return decryptedData;
}

Main();

# Asymmetric encryption

![Alt text](image-3.png)

# .NET надає безліч асиметричних алгоритмів і безліч реалізацій для деяких алгоритмів:

[DSA:](https://en.wikipedia.org/wiki/Digital_Signature_Algorithm) DSACng, DSACryptoServiceProvider, DSAOpenSsl

[ECDiffieHellman:](https://en.wikipedia.org/wiki/Curve25519) ECDiffieHellmanCng, ECDiffieHellmanOpenSsl

[ECDsa:](https://en.wikipedia.org/wiki/Elliptic_Curve_Digital_Signature_Algorithm) ECDsaCng, ECDsaOpenSsl

[RSA:](https://en.wikipedia.org/wiki/RSA_(cryptosystem)) RSACng, RSACryptoServiceProvider, RSAOpenSsl

Для шифрування даних можна використовувати RSA. Для довжини ключа слід використовувати щонайменше 2048 (NSA рекомендує 3072 або більше). Зверніть увагу, що RSA може шифрувати дані тільки до довжини ключа (включаючи пробіли). Отже, за допомогою ключа довжиною 2048 біт ви не зможете зашифрувати блок розміром 4096 біт. Вам потрібно буде розбити його на 2 блоки і зашифрувати їх окремо.

In [49]:
static void Main()
{
    var data = new byte[] { 1, 2, 3 };
    var (publicKey, privateKey) = GenerateKeys(2048);

    var encryptedData = Encrypt(data, publicKey);
    var decryptedData = Decrypt(encryptedData, privateKey);

    Console.WriteLine(GetRandomDataAsString(encryptedData));
    Console.WriteLine(GetRandomDataAsString(decryptedData));
}

static (RSAParameters publicKey, RSAParameters privateKey) GenerateKeys(int keyLength)
{
    using (var rsa = RSA.Create())
    {
        rsa.KeySize = keyLength;
        return (
            publicKey: rsa.ExportParameters(includePrivateParameters: false),
            privateKey: rsa.ExportParameters(includePrivateParameters: true)
        );
    }
}

static byte[] Encrypt(byte[] data, RSAParameters publicKey)
{
    using (var rsa = RSA.Create())
    {
        rsa.ImportParameters(publicKey);

        var result = rsa.Encrypt(data, RSAEncryptionPadding.OaepSHA256);
        return result;
    }
}

static byte[] Decrypt(byte[] data, RSAParameters privateKey)
{
    using (var rsa = RSA.Create())
    {
        rsa.ImportParameters(privateKey);
        return rsa.Decrypt(data, RSAEncryptionPadding.OaepSHA256);
    }
}

Main();

9C5983692474242D44BF2B6BA268CDBF06BC2D80229D4E5DDE923533F8D894FCD0B9A17EEC10808772478F4CE481AEDFF05F0C3457C33B26554E3ECAB5C40F7A1A6C334A9B0B4439D71AE84BAB833F00D7791E7DCA03983518972692F62774489D64762386F8A3C541E17DA8B22F122012F216562863A6BCB1620449D92B516BB1237559AB3F8387C87CDEC959179B87AB193FB1863711DC379BED87C4E5EB6B7FE506781DBCF43F0FE6141255EF0593F0B9DE71958602577185EE71CA584300F48B42E8ED386584EAD4A3299EDC45D83EA5574A646DDD59A6A5A08818C576612217D2B8DABAF96C14D4C0284D7429B5E4FFA834522B8557BA6AB36BF33A072D
010203


# Гібридне шифрування
Гібридне шифрування - це комбінація симетричних та асиметричних алгоритмів шифрування. Часто ви генеруєте симетричний ключ і шифруєте його за допомогою асиметричного алгоритму. Таким чином, ви використовуєте переваги асиметричного алгоритму для обміну ключем і можете використовувати швидкість AES порівняно з RSA для шифрування даних. Крім того, симетричний ключ досить малий, щоб його можна було зашифрувати за допомогою RSA. Зауважте, що ви також можете HMAC зашифровані дані за допомогою симетричного ключа, щоб перевірити цілісність даних.

# Цифровий підпис
Цифровий підпис дозволяє перевірити автентичність документа. Ви знаєте, хто підписав документ, і можете підтвердити, що документ не був змінений після підписання. Підпис працює подібно до асиметричного шифрування. Ви підписуєте документ (або його хеш) за допомогою свого приватного ключа. І будь-хто зможе перевірити цей підпис за допомогою вашого відкритого ключа. Якщо відкритий ключ пов'язаний з сертифікатом, ви зможете підтвердити особу людини, яка підписала документ.

In [52]:
static void Main()
{
    var (publicKey, privateKey) = GenerateKeys(2048);

    var data = new byte[] { 1, 2, 3 };
    var signedData = Sign(data, privateKey);
    var isValid = VerifySignature(data, signedData, publicKey);

    Console.WriteLine(isValid);
}

static (RSAParameters publicKey, RSAParameters privateKey) GenerateKeys(int keyLength)
{
    using (var rsa = RSA.Create())
    {
        rsa.KeySize = keyLength;
        return (
            publicKey: rsa.ExportParameters(includePrivateParameters: false),
            privateKey: rsa.ExportParameters(includePrivateParameters: true));
    }
}

static byte[] Sign(byte[] data, RSAParameters privateKey)
{
    using (var rsa = RSA.Create())
    {
        rsa.ImportParameters(privateKey);

        // the hash to sign
        byte[] hash;
        using (var sha256 = SHA256.Create())
        {
            hash = sha256.ComputeHash(data);
        }

        var rsaFormatter = new RSAPKCS1SignatureFormatter(rsa);
        rsaFormatter.SetHashAlgorithm("SHA256");
        return rsaFormatter.CreateSignature(hash);
    }
}

private static bool VerifySignature(byte[] data, byte[] signature, RSAParameters publicKey)
{
    using (var rsa = RSA.Create())
    {
        rsa.ImportParameters(publicKey);

        // the hash to sign
        byte[] hash;
        using (var sha256 = SHA256.Create())
        {
            hash = sha256.ComputeHash(data);
        }

        var rsaFormatter = new RSAPKCS1SignatureDeformatter(rsa);
        rsaFormatter.SetHashAlgorithm("SHA256");
        return rsaFormatter.VerifySignature(hash, signature);
    }
}

Main();

True


# Підпис XML-документів
Існує стандарт для підписання XML-документів. Він працює майже так само, як і загальний алгоритм підпису, за винятком того, що підпис може бути включений в документ. Ви також можете підписати лише частину документа, якщо це необхідно.

In [53]:
using System.Xml;
using System.Security.Cryptography.Xml;

static void Main()
{
    var (publicKey, privateKey) = GenerateKeys(2048);

    var document = new XmlDocument();
    document.LoadXml("<Root><Author>Meziantou</Author></Root>");

    SignXml(document, privateKey);
    var isValidXmlSignature = VerifyXmlSignature(document, publicKey);

    Console.WriteLine(isValidXmlSignature);
}

public static void SignXml(XmlDocument xmlDocument, RSAParameters privateKey)
{
    using (var rsa = RSA.Create())
    {
        rsa.ImportParameters(privateKey);

        var signedXml = new SignedXml(xmlDocument);
        signedXml.SigningKey = rsa;

        // Create a reference to be signed
        var reference = new Reference(""); // "" means entire document, https://learn.microsoft.com/en-us/dotnet/api/system.security.cryptography.xml.reference.uri?WT.mc_id=DT-MVP-5003978&view=dotnet-plat-ext-6.0
        var env = new XmlDsigEnvelopedSignatureTransform();
        reference.AddTransform(env);

        // Add the reference to the SignedXml object and compute the signature
        signedXml.AddReference(reference);
        signedXml.ComputeSignature();

        // Get the XML representation of the signature and add it to the document
        XmlElement xmlDigitalSignature = signedXml.GetXml();
        xmlDocument.DocumentElement.AppendChild(xmlDocument.ImportNode(xmlDigitalSignature, deep: true));
    }
}

public static bool VerifyXmlSignature(XmlDocument xmlDocument, RSAParameters publicKey)
{
    using (var rsa = RSA.Create())
    {
        rsa.ImportParameters(publicKey);

        var signatureElement = xmlDocument.GetElementsByTagName("Signature").OfType<XmlElement>().FirstOrDefault();
        var signedXml = new SignedXml(xmlDocument);
        signedXml.LoadXml(signatureElement);

        return signedXml.CheckSignature(rsa);
    }
}

Main();

True


# Here's the signed xml document:

In [ ]:
<Root>
  <Author>Meziantou</Author>
  <Signature xmlns="http://www.w3.org/2000/09/xmldsig#">
    <SignedInfo>
      <CanonicalizationMethod Algorithm="http://www.w3.org/TR/2001/REC-xml-c14n-20010315" />
      <SignatureMethod Algorithm="http://www.w3.org/2001/04/xmldsig-more#rsa-sha256" />
      <Reference URI="">
        <Transforms>
          <Transform Algorithm="http://www.w3.org/2000/09/xmldsig#enveloped-signature" />
        </Transforms>
        <DigestMethod Algorithm="http://www.w3.org/2001/04/xmlenc#sha256" />
        <DigestValue>/nqzc97wNrLZ03VLo8ycnAFEOduEHAyUeP4nnPaiWU8=</DigestValue>
      </Reference>
    </SignedInfo>
    <SignatureValue>MdtKVRg+esWEDv8+TqAt0XLWd7kzgWvluBk6i0IyirUMPnUKifnkA7DfRRKifXQagP+1jZ4LZ9dLNCzul1Y8w8ZZeE7dy40pdgYcppHl+1dq4qRnymz2yDwU3bg50ZoAAyLVcW3fn7AuN1QS4eOj5fhird9epIQQdVZz8f8hZMGHgpAhR+c2MFPW6EmzeAQ7XBrhMtc9GhIrwMCUczkSYFOHYp+jaTYPb8hfVvW2ACmApsKw5/a3uxQS/9+n4CTy4y5mdksjKZLRMOtLRlzStg4CSUnsYYsJK+1y3yfyQlIQuglTVi+K8yEX8ZI+C4jz8rTV3U5hbilNRZ3LMlVusA==</SignatureValue>
  </Signature>
</Root>

# Висновок
.NET Framework містить всі класи, необхідні для використання основних криптографічних алгоритмів. Більшість алгоритмів вимагають конфігурації. Обов'язково зверніться до документації, щоб дізнатися, які рекомендовані значення. Якщо вам потрібно використовувати алгоритм, який не передбачений фреймворком, обов'язково використовуйте зовнішню бібліотеку, яка дуже добре протестована і підтримується.